In [2]:
!pip install -q transformers

In [31]:
!pip install datasets

  Using cached datasets-2.13.2-py3-none-any.whl (512 kB)
  Using cached fsspec-2023.1.0-py3-none-any.whl (143 kB)
     |████████████████████████████████| 39.1 MB 133 kB/s            
  Using cached multiprocess-0.70.15-py37-none-any.whl (116 kB)
     |████████████████████████████████| 194 kB 252 kB/s            
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     |████████████████████████████████| 115 kB 242 kB/s            
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.11.0
    Uninstalling fsspec-2021.11.0:
      Successfully uninstalled fsspec-2021.11.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [1]:
from transformers import T5Tokenizer

# Initialize the tokenizer for the specific model (e.g., mBERT)
tokenizer = T5Tokenizer.from_pretrained('google/mt5-base')

# Sample text in two languages
text_english = "This is a sample English sentence."
text_spanish = "Esta es una oración de ejemplo en español."

# Tokenize the texts and get token IDs
token_ids_english = tokenizer.encode(text_english, add_special_tokens=True)
token_ids_spanish = tokenizer.encode(text_spanish, add_special_tokens=True)

# Convert token IDs to sets for easy intersection calculation
token_set_english = set(token_ids_english)
token_set_spanish = set(token_ids_spanish)

# Calculate token intersection
token_intersection = token_set_english.intersection(token_set_spanish)

# Print the number of tokens in the intersection
print("Number of tokens in the intersection:", len(token_intersection))


Number of tokens in the intersection: 3


# Start

In [8]:
import os
import pandas as pd
from datasets import load_dataset
from pathlib import Path
from tqdm.auto import tqdm
import pickle

In [9]:
path_to_dataset = "/home/jovyan/datasets/XL_Dataset/jovyan/protasov/XL_Dataset/"

In [10]:
all_languages = os.listdir(path_to_dataset)

df = pd.read_csv("../LR_Transfer/data/Collected_langs.csv")
hr_languages = df[(df.N_tokens >= 350000)].Name.tolist()
lr_languages = df[(df.N_tokens > 10000) & (df.N_tokens < 350000)].Name.tolist()

In [4]:
# check all languages are presented in our dataset
for i in df.Name.tolist():
    assert i in all_languages, i

In [7]:
langs = ['Yoruba',
'Acehnese',
'Qafar',
'Ossetic',
'Zhuang (Northern)',
'Uzbek',
'Bislama',
'Bugis',
'Cherokee',
'Cheyenne',
'Choctaw',
'Fuzhou',
'Arabic (Egyptian)',
'Gorontalo',
'Sotho (Northern)',
'Hakka',
'Kikuyu',
'Kongo',
'Meithei',
'Seminole',
'Nauruan',
'Yi',
'Pangasinan',
'Sango',
'Tahitian',
'Tok Pisin',
'Waray-Waray',
'Wu',
'Zulu',
'Crimean Tatar']


lr_n = 0
hr_n = 0

for i in langs:
    if i in hr_languages:
        hr_n += 1
    elif i in lr_languages:
        lr_n += 1
        
        
lr_n, hr_n, len(langs)

(10, 16, 30)

In [5]:
# %%time

# datasets_by_langs_path = "datasets_by_langs.pickle"
# with open(datasets_by_langs_path, 'rb') as f:
#     x = pickle.load(f)

# datasets_by_langs_path2 = "datasets_by_langs_2.pickle"
# with open(datasets_by_langs_path2, 'rb') as f:
#     x2 = pickle.load(f)
    
# datasets_by_langs_path3 = "datasets_by_langs_3.pickle"
# with open(datasets_by_langs_path3, 'rb') as f:
#     x3 = pickle.load(f)
    
# datasets_by_langs_path4 = "datasets_by_langs_4.pickle"
# with open(datasets_by_langs_path4, 'rb') as f:
#     x4 = pickle.load(f)
    
# datasets_by_langs_path5 = "datasets_by_langs_5.pickle"
# with open(datasets_by_langs_path5, 'rb') as f:
#     x5 = pickle.load(f)

# datasets_by_langs_path6 = "datasets_by_langs_6.pickle"
# with open(datasets_by_langs_path6, 'rb') as f:
#     x6 = pickle.load(f)
    
# datasets_by_langs_path7 = "datasets_by_langs_7.pickle"
# with open(datasets_by_langs_path7, 'rb') as f:
#     x7 = pickle.load(f)

# datasets_by_langs_path8 = "datasets_by_langs_8.pickle"
# with open(datasets_by_langs_path8, 'rb') as f:
#     x8 = pickle.load(f)

In [6]:
# processed_langs = list(x.keys()) + list(x2.keys()) + list(x3.keys()) \
# + list(x4.keys()) + list(x5.keys()) + list(x6.keys()) + list(x7.keys()) + list(x8.keys())

In [10]:
error_langs = ["Russian", "Slovene", "Solon", "Spanish",
               "Swedish (Västerbotten)", "Tajik", "Thai", "Turkmen",
               "Tibetan (Modern Literary)", "Tamil (Spoken)", "Tulu", "Turkish", "Tuvan",
               "Tatar", "Udmurt", "Ukrainian"
              ]

In [ ]:
datasets_by_langs = {}
for lang in tqdm(["Arabic (Egyptian)"], desc="Processing languages"):
    lang_paths = [str(p) for p in list(Path(path_to_dataset, lang).glob("**/*"))]

    lang_data = []

    for path in tqdm(lang_paths, desc=f"Processing files of {lang}"):
        res = load_dataset('text', data_files=path, split='train', num_proc=1)
        lang_data += res["text"]

    datasets_by_langs[lang] = lang_data

    # Store data (serialize)
    with open("datasets_by_langs_all.pickle", 'wb') as handle:
        pickle.dump(datasets_by_langs, handle, protocol=pickle.HIGHEST_PROTOCOL)

Processing languages:   0%|          | 0/1 [00:00<?, ?it/s]

Processing files of Arabic (Egyptian):   0%|          | 0/485 [00:00<?, ?it/s]

Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-edfbc8a33edc8ef4/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-d7b0616e1b4aad9d/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-014aa7db2e7cf0c5/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-d1290f5dfd148924/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-1e9be0c38043adb6/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-9a6c04512d7026a0/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5

Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-f7e68d0528f3c050/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-e51ca59dfc4efa68/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-128370c9d79363ae/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-1fa5c71fd6c8d3b3/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-04dbc3885dc1c7cb/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-2072259db09a601b/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5

Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-2dbcd937b3bc51dd/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-ea536d00de9d6e68/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-55d8698375c1e688/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-712fb6970af7d3bd/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-8e92ccb0a447fd8c/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-6706a94bb4c5ee4f/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/text/default-27e8f61a76784e7c/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
res["text"]

KeyError: 'text'

In [ ]:
datasets_by_langs = {}
for lang in tqdm(hr_languages+lr_languages, desc="Processing languages"):
    lang_paths = [str(p) for p in list(Path(path_to_dataset, lang).glob("**/*"))]

    lang_data = []

    for path in tqdm(lang_paths, desc=f"Processing files of {lang}"):
        lang_data += load_dataset('text', data_files=path, split='train')["text"]

    datasets_by_langs[lang] = lang_data

    # Store data (serialize)
    with open("datasets_by_langs_all.pickle", 'wb') as handle:
        pickle.dump(datasets_by_langs, handle, protocol=pickle.HIGHEST_PROTOCOL)

Processing languages:   0%|          | 0/261 [00:00<?, ?it/s]

Processing files of Abaza:   0%|          | 0/29 [00:00<?, ?it/s]

Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-489e16507e42da30/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-9335bda0c5b6ccef/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-8563c9ddc5d218db/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-dd07451921e37f5a/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-370f8227580d5164/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
Found cached dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-23f8ffa651db4872/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/text/default-0524007324e3c5c4/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def flatten_list_of_lists(lst: list) -> list:
    """Flatten a list of lists into a single list."""
    return [item for sublist in lst for item in sublist]

def find_unique_elements(lst: list) -> set:
    """Find unique elements from a list."""
    flattened_list = flatten_list_of_lists(lst)
    return set(flattened_list)